In [1]:
!pip install efficientnet

import os
os.chdir("../")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from efficientnet.keras import EfficientNetB5

from keras.models import Sequential
from keras import layers
from keras.optimizers import Adam

from keras.callbacks import ReduceLROnPlateau

import tensorflow as tf
from tensorflow.keras import backend as K


img_list = np.load("../data/processed/img_list.npz", allow_pickle=True)
img_names_list = np.load("../data/processed/image_names_list.npz", allow_pickle=True)
img_list = img_list['arr_0']
img_names_list= img_names_list['arr_0']

df = pd.read_csv("../data/files/train.csv")

df_dict = dict()
for line in range(len(df)):
  df_dict[df["id_code"][line]] = {"diagnosis" : df["diagnosis"][line], "array" : None}

for image_name, img in zip(img_names_list, img_list):
  image_name = image_name.strip(".png")
  df_dict[image_name]["array"] = img


img_list = list()
y_train = list()

for image_name, img_data in df_dict.items():
  label = img_data["diagnosis"]
  image_array = img_data["array"]

  y_train.append(label)
  img_list.append(image_array)


# GPU kullanımı için TensorFlow'u yapılandırma
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)

y_train = pd.get_dummies(y_train).values
#y_train = pd.get_dummies(df["diagnosis"]).values
y_train_son = np.ones(y_train.shape, dtype="uint8")
y_train_son[:,4] = y_train[:,4]

for i in range(3,-1,-1):
    y_train_son[:,i] = np.logical_or(y_train[:,i], y_train_son[:,i+1])

img_list = np.array(img_list)
print(img_list.shape)

img_list, x_val, y_train, y_val = train_test_split(img_list,
                                                  y_train_son,
                                                  test_size=0.15,
                                                  random_state=2019,
                                                  shuffle=True)

datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True)
data_generator = datagen.flow(img_list, y_train, batch_size=2, seed=2020)

model = Sequential()
model.add(EfficientNetB5(weights="imagenet", include_top=False, input_shape=(400, 400, 3)))
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(5, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=0.00005), metrics=["accuracy"])

lr = ReduceLROnPlateau(monitor="val_loss",  # hatalı olan "val_los" -> "val_loss"
                       patience=3,
                       verbose=1,
                       mode="auto",
                       factor=0.25,
                       min_lr=0.000001)

for epoch_amount in [30,50,100]:
  history = model.fit(data_generator,
                      steps_per_epoch=1000,
                      epochs=epoch_amount,
                      validation_data=(x_val, y_val),
                      callbacks=[lr])
  model.save(f'../saved_models/diabetic_eye_model_GPU_trained_{epoch_amount}_epoch.h5')